<a href="https://colab.research.google.com/github/baeksumin/k-project/blob/main/data_preprocessing/optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install backtrader

     |████████████████████████████████| 410 kB 5.1 MB/s 


In [ ]:
!pip install backtesting

     |████████████████████████████████| 175 kB 5.4 MB/s 
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173823 sha256=c595d185e46055af48853b12f220198302d1445ad938402b1dd9eacc30aee0de
  Stored in directory: /root/.cache/pip/wheels/48/1d/ea/65dacebc37da7655d8a1fd0f315ac39d102e31d6545237a1c6
Successfully built backtesting


In [ ]:
!pip install deap

     |████████████████████████████████| 160 kB 5.4 MB/s 


In [ ]:
#talib 설치
!wget -q http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz 2>&1 > /dev/null
!tar xvzf ta-lib-0.4.0-src.tar.gz 2>&1 > /dev/null

import os

os.chdir('ta-lib')  

!./configure --prefix=/usr 2>&1 > /dev/null
!make 2>&1 > /dev/null
!make install 2>&1 > /dev/null

os.chdir('../')

!pip install TA-Lib 2>&1 > /dev/null

./configure: line 4354: /usr/bin/file: No such file or directory
libtool: link: warning: `-version-info/-version-number' is ignored for convenience libraries
ar: `u' modifier ignored since `D' is the default (see `U')
libtool: link: warning: `-version-info/-version-number' is ignored for convenience libraries
ar: `u' modifier ignored since `D' is the default (see `U')
libtool: link: warning: `-version-info/-version-number' is ignored for convenience libraries
ar: `u' modifier ignored since `D' is the default (see `U')
libtool: link: warning: `-version-info/-version-number' is ignored for convenience libraries
ar: `u' modifier ignored since `D' is the default (see `U')
ar: `u' modifier ignored since `D' is the default (see `U')
gen_code.c: In function ‘printFuncHeaderDoc’:
gen_code.c:3456:4: warning: format not a string literal and no format arguments [-Wformat-security]
    fprintf( out, prefix );
    ^~~~~~~


In [ ]:
import re
import sys
import json
import time
import talib
import pickle
import random
import logging
import os.path
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import trange
import backtrader as bt
from google.colab import drive
import matplotlib.pyplot as plt
from backtesting import Strategy
from backtesting import Backtest
import backtrader.feeds as btfeeds
from IPython.display import display, Image
from datetime import datetime, date, timedelta
from deap import base, creator, tools, algorithms

pd.options.mode.chained_assignment = None  # default='warn'

/usr/local/lib/python3.7/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [ ]:
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/[Shared]K-Project/data/'

Mounted at /content/drive


In [ ]:
## Load pickle - fullcode 목록
with open(data_path + 'full_code.pkl',"rb") as f: 
    fullcode = pickle.load(f)
    
# 상장폐지 상태 제외시키기
fullcode_filter = fullcode[fullcode['status'] != '상장폐지']

# fullcode열을 list로 변경
fullcode_list = list(fullcode_filter['full_code'])
fullcode_list0 = fullcode_list[:1]
# fullcode_list0 = fullcode_list[260:520]
# fullcode_list0 = fullcode_list[520:780]
# fullcode_list0 = fullcode_list[780:1040]
# fullcode_list0 = fullcode_list[1040:1300]
# fullcode_list0 = fullcode_list[1300:1560]
# fullcode_list0 = fullcode_list[1560:1820]
# fullcode_list0 = fullcode_list[1820:2080]
# fullcode_list0 = fullcode_list[2080:2340]
# fullcode_list0 = fullcode_list[2340:]
print(len(fullcode_list0))

1


In [ ]:
with open(data_path + 'initial_data1.json',"r") as json_file:
    json_data = json.load(json_file) 

In [ ]:
# 전략

class SmaCross1(bt.Strategy):
  params = dict(
    pfast=50, # period for the fast moving average
    pslow=200 # period for the slow moving average 
    ) 
  
  def __init__(self):
    sma1 = bt.ind.SMA(period = self.p.pfast) # fast moving average 
    sma2 = bt.ind.SMA(period = self.p.pslow) # slow moving average 
    self.crossover = bt.ind.CrossOver(sma1, sma2) # crossover signal 

  def next(self): 
    if not self.position: # not in the market 
      if self.crossover > 0: # if fast crosses slow to the upside 
        close = self.data.close[0] # 종가 값 
          
        size = int(self.broker.getcash() / close) # 최대 구매 가능 개수 
        self.buy(size=size) # 매수 size = 구매 개수 설정 
    elif self.crossover < 0: # in the market & cross to the downside 
        self.close() # 매도

class RSI(bt.Strategy):
  params = dict(period=26)

  def __init__(self):
    self.rsi = bt.indicators.RSI(self.data.close, period=self.p.period)

  def next(self):    
    if not self.position:  #아직 주식을 사지 않았다면

      if self.rsi <30 :
        self.order = self.buy()

    elif self. rsi >70 :
      self.order = self.sell()

class ROC(bt.Strategy):
  params = dict(period=14)

  def __init__(self):
    self.roc = bt.indicators.ROC(self.data.close, period=self.p.period)

  def next(self):    
    if not self.position:  #아직 주식을 사지 않았다면

      if self.roc > 0:
        self.order = self.buy()

    elif self.roc < 0:
      self.order = self.sell()

class EmaCross(bt.Strategy):
    params = dict(
        period_1=12,
        period_2=26,
        period_signal=9
    )
    
    ## 초기화 메서드
    def __init__(self):
        ema1 = bt.ind.EMA(self.data, period=self.p.period_1) # 단기 이동 평균선
        ema2 = bt.ind.EMA(self.data, period=self.p.period_2) # 장기 이동 평균선
        self.crossover = bt.ind.CrossOver(ema1, ema2)
        
    def next(self):
        if not self.position:
            if self.crossover > 0:  # 단기 이동 평균선이 장기 이동 평균선을 상향 돌파시 매수
                self.buy()
                
        elif self.crossover < 0:  # 단기 이동 평균선이 장기 이동 평균선을 하향 돌파시 매수
            self.close()

class StochasticSR(bt.Strategy):
    '''Trading strategy that utilizes the Stochastic Oscillator indicator for oversold/overbought entry points, 
    and previous support/resistance via Donchian Channels as well as a max loss in pips for risk levels.'''
    # parameters for Stochastic Oscillator and max loss in pips
    # Donchian Channels to determine previous support/resistance levels will use the given period as well
    # http://www.ta-guru.com/Book/TechnicalAnalysis/TechnicalIndicators/Stochastic.php5 for Stochastic Oscillator formula and description
    params = (('period', 14), ('pfast', 3), ('pslow', 3), ('upperLimit', 80), ('lowerLimit', 20), ('stop_pips', .002))

    def __init__(self):
        '''Initializes logger and variables required for the strategy implementation.'''
        # initialize logger for log function (set to critical to prevent any unwanted autologs, not using log objects because only care about logging one thing)
        for handler in logging.root.handlers[:]:
            logging.root.removeHandler(handler)

        logging.basicConfig(format='%(message)s', level=logging.CRITICAL, handlers=[
            logging.FileHandler("LOG.log"),
            logging.StreamHandler()
            ])

        self.order = None
        self.donchian_stop_price = None
        self.price = None
        self.stop_price = None
        self.stop_donchian = None

        self.stochastic = bt.indicators.Stochastic(self.data, period=self.params.period, period_dfast=self.params.pfast, period_dslow=self.params.pslow, 
        upperband=self.params.upperLimit, lowerband=self.params.lowerLimit)


    def next(self):
        '''Checks to see if Stochastic Oscillator, position, and order conditions meet the entry or exit conditions for the execution of buy and sell orders.'''
        if self.order:
            # if there is a pending order, don't do anything
            return
        if self.position.size == 0:
            # When stochastic crosses back below 80, enter short position.
            if self.stochastic.lines.percD[-1] >= 80 and self.stochastic.lines.percD[0] <= 80:
                # stop price at last support level in self.params.period periods
                self.donchian_stop_price = max(self.data.high.get(size=self.params.period))
                self.order = self.sell()
                # stop loss order for max loss of self.params.stop_pips pips
                self.stop_price = self.buy(exectype=bt.Order.Stop, price=self.data.close[0]+self.params.stop_pips, oco=self.stop_donchian)
                # stop loss order for donchian SR price level
                self.stop_donchian = self.buy(exectype=bt.Order.Stop, price=self.donchian_stop_price, oco=self.stop_price)
            # when stochastic crosses back above 20, enter long position.
            elif self.stochastic.lines.percD[-1] <= 20 and self.stochastic.lines.percD[0] >= 20:
                # stop price at last resistance level in self.params.period periods
                self.donchian_stop_price = min(self.data.low.get(size=self.params.period))
                self.order = self.buy()
                # stop loss order for max loss of self.params.stop_pips pips
                self.stop_price = self.sell(exectype=bt.Order.Stop, price=self.data.close[0]-self.params.stop_pips, oco=self.stop_donchian)
                # stop loss order for donchian SR price level
                self.stop_donchian = self.sell(exectype=bt.Order.Stop, price=self.donchian_stop_price, oco=self.stop_price) 
  
        if self.position.size > 0:
            # When stochastic is above 70, close out of long position
            if (self.stochastic.lines.percD[0] >= 70):
                self.close(oco=self.stop_price)
        if self.position.size < 0:
            # When stochastic is below 30, close out of short position
            if (self.stochastic.lines.percD[0] <= 30):
                self.close(oco=self.stop_price)

In [ ]:
result_dict = {}

for j in range(len(fullcode_list0)): 

  print(j)
  print(fullcode_list0[j])
  
  # backtest 데이터 가공

  df = pd.read_json(json_data[fullcode_list0[j]], orient ='index') # 첫번째 키값으로 데이터프레임 변환
  df = df.transpose()

  #시간순 재정렬
  df = df.sort_values(by=['TRD_DD'])
  df.reset_index(drop=True,inplace=True)
  df['TRD_DD'] = pd.to_datetime(df['TRD_DD']) #datetime변환

  #인풋 데이터 모양 맞춰주기(backtest에 들어갈 데이터 모양)
  df_bt = df[['TRD_DD','TDD_OPNPRC','TDD_HGPRC','TDD_LWPRC','TDD_CLSPRC', 'ACC_TRDVOL']]
  df_bt['TRD_DD'] = pd.to_datetime(df_bt['TRD_DD'])
  df_bt.rename(columns={'TRD_DD':'Date', 'TDD_OPNPRC':'Open', 'TDD_HGPRC':'High','TDD_LWPRC':'Low','TDD_CLSPRC':'Close', 'ACC_TRDVOL':'Volume'}, inplace=True)
  df_bt.set_index('Date',drop=True,inplace=True)

  #데이터프레임 콤마(,) 제거 그리고 타입 소수로 변환
  df_bt['Open'] = df_bt['Open'].str.replace(',','').astype('float')
  df_bt['High'] = df_bt['High'].str.replace(',','').astype('float')
  df_bt['Low'] = df_bt['Low'].str.replace(',','').astype('float')
  df_bt['Close'] = df_bt['Close'].str.replace(',','').astype('float')
  df_bt['Volume'] = df_bt['Volume'].str.replace(',','').astype('float')


# GDC --------------------------------------------------------------------------
  random.seed(3)

  PARAM_NAMES = ["pfast", "pslow"]

  NGEN = 5  # 알고리즘 5번 반복.
  NPOP = 100 #인구 초기
  CXPB = 0.5  #교차 전략 
  MUTPB = 0.3  #돌연변이 전략.


  #최소fintness 설정 (fitness값이 작을수록 좋도록 설정)
  creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
  creator.create('Individual', list, fitness=creator.FitnessMin)

  # creator.create("FitnessMax", base.Fitness, weights=(1.0,))
  # creator.create("Individual", list, fitness=creator.FitnessMax)

  def evaluate(individual, plot=False, log=False):

    strategy_params = {k: v for k, v in zip(PARAM_NAMES, individual)}

    cerebro = bt.Cerebro(stdstats=False)

    data = bt.feeds.PandasData(dataname = df_bt)

    cerebro.adddata(data)

    initial_capital = 1000000
    cerebro.broker.setcash(initial_capital)

    cerebro.addstrategy(SmaCross1, **strategy_params)

    cerebro.addanalyzer(bt.analyzers.DrawDown)

    cerebro.broker.setcommission(commission=0.0025, margin=False)  #수수료 설정

    strats = cerebro.run()

    profit = cerebro.broker.getvalue() - initial_capital
    
    if profit == 0:
      return [np.inf]

    # max_dd = strats[0].analyzers.drawdown.get_analysis()["max"]["moneydown"] # max.moneydown - max drawdown value in monetary units
    # fitness = profit / (max_dd if max_dd > 0 else 1)
    fitness = round(1 / profit, 15)

    if log:
      print(f"Starting Portfolio Value: {initial_capital:,.2f}")
      print(f"Final Portfolio Value:  {cerebro.broker.getvalue():,.2f}")
      print(f"Total Profit:       {profit:,.2f}")
      print(f"Profit / Max DD:     {fitness}")

    # if plot:
      # cerebro.plot()

    return [fitness]

  toolbox = base.Toolbox()
  toolbox.register("indices", random.sample, range(NPOP), NPOP)

  # crossover strategy
  toolbox.register("mate", tools.cxUniform, indpb=CXPB)
  # mutation strategy
  toolbox.register("mutate", tools.mutUniformInt, low=1, up=151, indpb=0.2)
  # selection strategy
  toolbox.register("select", tools.selTournament, tournsize=3)
  # fitness function
  toolbox.register("evaluate", evaluate)


  # definition of an individual & a population
  toolbox.register("attr_sma1", random.randint, 1, 100)
  toolbox.register("attr_sma2", random.randint, 151, 251) 
  toolbox.register(
    "individual",
    tools.initCycle,
    creator.Individual,
    (
      toolbox.attr_sma1,
      toolbox.attr_sma2,

    ),
  )

  toolbox.register("population", tools.initRepeat, list, toolbox.individual)

  mean = np.ndarray(NGEN)
  best = np.ndarray(NGEN)
  hall_of_fame = tools.HallOfFame(maxsize=3)

  t = time.perf_counter()
  pop = toolbox.population(n=NPOP)
  for g in trange(NGEN):
    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
      if random.random() < CXPB:
        toolbox.mate(child1, child2)
        del child1.fitness.values
        del child2.fitness.values

    # Apply mutation on the offspring
    for mutant in offspring:
      if random.random() < MUTPB:
        toolbox.mutate(mutant)
        del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
      ind.fitness.values = fit

    # The population is entirely replaced by the offspring
    pop[:] = offspring
    hall_of_fame.update(pop)
    print(
      "\n HALL OF FAME:\n"
      + "\n".join(
        [
          f"  {_}: {ind}, Fitness: {ind.fitness.values[0]}"
          for _, ind in enumerate(hall_of_fame)
        ]
      )
    )

    fitnesses = [
      ind.fitness.values[0] for ind in pop if not np.isinf(ind.fitness.values[0])
    ]
    mean[g] = np.mean(fitnesses)
    best[g] = np.max(fitnesses)

  end_t = time.perf_counter()
  print(f"Time Elapsed: {end_t - t:,.2f}")
  
  # 최적의 파라미터 값 출력
  OPTIMISED_STRATEGY_PARAMS = {
    k: v for k, v in zip(PARAM_NAMES, hall_of_fame[0])}
  GDC_params = list(OPTIMISED_STRATEGY_PARAMS.values())
  print('**GDC 파라미터 값: ', GDC_params)
  print('\n')
  
# RSI --------------------------------------------------------------------------

  random.seed(3)

  PARAM_NAMES = ["period"]

  NGEN = 5  # 알고리즘 5번 반복.
  NPOP = 100 #인구 초기
  CXPB = 0.5  #교차 전략 
  MUTPB = 0.3  #돌연변이 전략.


  #최소fintness 설정 (fitness값이 작을수록 좋도록 설정)
  creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
  creator.create('Individual', list, fitness=creator.FitnessMin)

  # creator.create("FitnessMax", base.Fitness, weights=(1.0,))
  # creator.create("Individual", list, fitness=creator.FitnessMax)

  def evaluate(individual, plot=False, log=False):

    strategy_params = {k: v for k, v in zip(PARAM_NAMES, individual)}

    cerebro = bt.Cerebro(stdstats=False)

    data = bt.feeds.PandasData(dataname = df_bt)

    cerebro.adddata(data)

    initial_capital = 1000000
    cerebro.broker.setcash(initial_capital)

    cerebro.addstrategy(RSI, **strategy_params)

    cerebro.addanalyzer(bt.analyzers.DrawDown)

    cerebro.broker.setcommission(commission=0.0025, margin=False)  #수수료 설정

    strats = cerebro.run()

    profit = cerebro.broker.getvalue() - initial_capital
    
    if profit == 0:
      return [np.inf]

    # max_dd = strats[0].analyzers.drawdown.get_analysis()["max"]["moneydown"] # max.moneydown - max drawdown value in monetary units
    # fitness = profit / (max_dd if max_dd > 0 else 1)
    fitness = round(1 / profit, 15)

    if log:
      print(f"Starting Portfolio Value: {initial_capital:,.2f}")
      print(f"Final Portfolio Value:  {cerebro.broker.getvalue():,.2f}")
      print(f"Total Profit:       {profit:,.2f}")
      print(f"Profit / Max DD:     {fitness}")

    # if plot:
      # cerebro.plot()

    return [fitness]

  toolbox = base.Toolbox()
  toolbox.register("indices", random.sample, range(NPOP), NPOP)

  # crossover strategy
  toolbox.register("mate", tools.cxUniform, indpb=CXPB)
  # mutation strategy
  toolbox.register("mutate", tools.mutUniformInt, low=1, up=151, indpb=0.2)
  # selection strategy
  toolbox.register("select", tools.selTournament, tournsize=3)
  # fitness function
  toolbox.register("evaluate", evaluate)


  # definition of an individual & a population
  toolbox.register("attr_period", random.randint, 1, 100)
  toolbox.register(
    "individual",
    tools.initCycle,
    creator.Individual,
    (
      toolbox.attr_period,

    ),
  )

  toolbox.register("population", tools.initRepeat, list, toolbox.individual)

  mean = np.ndarray(NGEN)
  best = np.ndarray(NGEN)
  hall_of_fame = tools.HallOfFame(maxsize=3)

  t = time.perf_counter()
  pop = toolbox.population(n=NPOP)
  for g in trange(NGEN):
    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
      if random.random() < CXPB:
        toolbox.mate(child1, child2)
        del child1.fitness.values
        del child2.fitness.values

    # Apply mutation on the offspring
    for mutant in offspring:
      if random.random() < MUTPB:
        toolbox.mutate(mutant)
        del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
      ind.fitness.values = fit

    # The population is entirely replaced by the offspring
    pop[:] = offspring
    hall_of_fame.update(pop)
    print(
      "\n HALL OF FAME:\n"
      + "\n".join(
        [
          f"  {_}: {ind}, Fitness: {ind.fitness.values[0]}"
          for _, ind in enumerate(hall_of_fame)
        ]
      )
    )

    fitnesses = [
      ind.fitness.values[0] for ind in pop if not np.isinf(ind.fitness.values[0])
    ]
    mean[g] = np.mean(fitnesses)
    best[g] = np.max(fitnesses)

  end_t = time.perf_counter()
  print(f"Time Elapsed: {end_t - t:,.2f}")

  # 최적의 파라미터 값 출력
  OPTIMISED_STRATEGY_PARAMS = {
    k: v for k, v in zip(PARAM_NAMES, hall_of_fame[0])}
  RSI_params = list(OPTIMISED_STRATEGY_PARAMS.values())
  print('RSI 파라미터 값: ', RSI_params)
  print('\n')

# ROC --------------------------------------------------------------------------

  random.seed(3)

  PARAM_NAMES = ["period"]

  NGEN = 5  # 알고리즘 5번 반복.
  NPOP = 100 #인구 초기
  CXPB = 0.5  #교차 전략 
  MUTPB = 0.3  #돌연변이 전략.


  #최소fintness 설정 (fitness값이 작을수록 좋도록 설정)
  creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
  creator.create('Individual', list, fitness=creator.FitnessMin)

  # creator.create("FitnessMax", base.Fitness, weights=(1.0,))
  # creator.create("Individual", list, fitness=creator.FitnessMax)

  def evaluate(individual, plot=False, log=False):

    strategy_params = {k: v for k, v in zip(PARAM_NAMES, individual)}

    cerebro = bt.Cerebro(stdstats=False)

    data = bt.feeds.PandasData(dataname = df_bt)

    cerebro.adddata(data)

    initial_capital = 1000000
    cerebro.broker.setcash(initial_capital)

    cerebro.addstrategy(ROC, **strategy_params)

    cerebro.addanalyzer(bt.analyzers.DrawDown)

    cerebro.broker.setcommission(commission=0.0025, margin=False)  #수수료 설정

    strats = cerebro.run()

    profit = cerebro.broker.getvalue() - initial_capital

    if profit == 0:
      return [np.inf]

    # max_dd = strats[0].analyzers.drawdown.get_analysis()["max"]["moneydown"] # max.moneydown - max drawdown value in monetary units
    # fitness = profit / (max_dd if max_dd > 0 else 1)
    fitness = round(1 / profit, 15)

    if log:
      print(f"Starting Portfolio Value: {initial_capital:,.2f}")
      print(f"Final Portfolio Value:  {cerebro.broker.getvalue():,.2f}")
      print(f"Total Profit:       {profit:,.2f}")
      print(f"Profit / Max DD:     {fitness}")

    # if plot:
      # cerebro.plot()

    return [fitness]

  toolbox = base.Toolbox()
  toolbox.register("indices", random.sample, range(NPOP), NPOP)

  # crossover strategy
  toolbox.register("mate", tools.cxUniform, indpb=CXPB)
  # mutation strategy
  toolbox.register("mutate", tools.mutUniformInt, low=1, up=151, indpb=0.2)
  # selection strategy
  toolbox.register("select", tools.selTournament, tournsize=3)
  # fitness function
  toolbox.register("evaluate", evaluate)


  # definition of an individual & a population
  toolbox.register("attr_period", random.randint, 1, 100)
  toolbox.register(
    "individual",
    tools.initCycle,
    creator.Individual,
    (
      toolbox.attr_period,

    ),
  )

  toolbox.register("population", tools.initRepeat, list, toolbox.individual)

  mean = np.ndarray(NGEN)
  best = np.ndarray(NGEN)
  hall_of_fame = tools.HallOfFame(maxsize=3)

  t = time.perf_counter()
  pop = toolbox.population(n=NPOP)
  for g in trange(NGEN):
    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
      if random.random() < CXPB:
        toolbox.mate(child1, child2)
        del child1.fitness.values
        del child2.fitness.values

    # Apply mutation on the offspring
    for mutant in offspring:
      if random.random() < MUTPB:
        toolbox.mutate(mutant)
        del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
      ind.fitness.values = fit

    # The population is entirely replaced by the offspring
    pop[:] = offspring
    hall_of_fame.update(pop)
    print(
      "\n HALL OF FAME:\n"
      + "\n".join(
        [
          f"  {_}: {ind}, Fitness: {ind.fitness.values[0]}"
          for _, ind in enumerate(hall_of_fame)
        ]
      )
    )

    fitnesses = [
      ind.fitness.values[0] for ind in pop if not np.isinf(ind.fitness.values[0])
    ]
    mean[g] = np.mean(fitnesses)
    best[g] = np.max(fitnesses)

  end_t = time.perf_counter()
  print(f"Time Elapsed: {end_t - t:,.2f}")
  
  # 최적의 파라미터 값 출력
  OPTIMISED_STRATEGY_PARAMS = {
    k: v for k, v in zip(PARAM_NAMES, hall_of_fame[0])}
  ROC_params = list(OPTIMISED_STRATEGY_PARAMS.values())
  print('**ROC 파라미터 값: ', ROC_params)

# MAE --------------------------------------------------------------------------

  # random.seed(3)

  # PARAM_NAMES = ["period_1", "period_2", "period_signal"]

  # NGEN = 5   # 알고리즘 5번 반복.
  # NPOP = 100  #인구 초기
  # CXPB = 0.5   #교차 전략 
  # MUTPB = 0.3   #돌연변이 전략.


  # #최소fintness 설정.(fitness값이 작을수록 좋도록 설정.)
  # #creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
  # #creator.create('Individual', list, fitness=creator.FitnessMin)

  # creator.create("FitnessMax", base.Fitness, weights=(1.0,))
  # creator.create("Individual", list, fitness=creator.FitnessMax)

  # def evaluate(individual, plot=False, log=False):

  #   strategy_params = {k: v for k, v in zip(PARAM_NAMES, individual)}

  #   cerebro = bt.Cerebro(stdstats=False)

  #   data = bt.feeds.PandasData(dataname = df_bt)

  #   cerebro.adddata(data)

  #   initial_capital = 1000000
  #   cerebro.broker.setcash(initial_capital)

  #   cerebro.addstrategy(RSI, **strategy_params)

  #   cerebro.addanalyzer(bt.analyzers.DrawDown)

  #   cerebro.broker.setcommission(commission=0.0025, margin=False)   #수수료 설정.

  #   strats = cerebro.run()

  #   profit = cerebro.broker.getvalue() - initial_capital

  #   if profit == 0:
  #     return [np.inf]

  #   # max_dd = strats[0].analyzers.drawdown.get_analysis()["max"]["moneydown"]
  #   # fitness = profit / (max_dd if max_dd > 0 else 1)
  #   fitness = round(1 / profit, 15)

  #   if log:
  #     print(f"Starting Portfolio Value: {initial_capital:,.2f}")
  #     print(f"Final Portfolio Value:    {cerebro.broker.getvalue():,.2f}")
  #     print(f"Total Profit:             {profit:,.2f}")
  #     print(f"Profit / Max DD:          {fitness}")

  #   if plot:
  #     cerebro.plot()

  #   return [fitness]

  # toolbox = base.Toolbox()
  # toolbox.register("indices", random.sample, range(NPOP), NPOP)

  # # crossover strategy
  # toolbox.register("mate", tools.cxUniform, indpb=CXPB)
  # # mutation strategy
  # toolbox.register("mutate", tools.mutUniformInt, low=1, up=151, indpb=0.2)
  # # selection strategy
  # toolbox.register("select", tools.selTournament, tournsize=3)
  # # fitness function
  # toolbox.register("evaluate", evaluate)


  # # definition of an individual & a population
  # toolbox.register("attr_period", random.randint, 1, 100)  #기간 1~100 범위 설정.
  # toolbox.register(
  #     "individual",
  #     tools.initCycle,
  #     creator.Individual,
  #     (
  #         toolbox.attr_period,

  #     ),
  # )

  # toolbox.register("population", tools.initRepeat, list, toolbox.individual)

  # mean = np.ndarray(NGEN)
  # best = np.ndarray(NGEN)
  # hall_of_fame = tools.HallOfFame(maxsize=3)

  # t = time.perf_counter()
  # pop = toolbox.population(n=NPOP)
  # for g in trange(NGEN):
  #     # Select the next generation individuals
  #     offspring = toolbox.select(pop, len(pop))
  #     # Clone the selected individuals
  #     offspring = list(map(toolbox.clone, offspring))

  #     # Apply crossover on the offspring
  #     for child1, child2 in zip(offspring[::2], offspring[1::2]):
  #         if random.random() < CXPB:
  #             toolbox.mate(child1, child2)
  #             del child1.fitness.values
  #             del child2.fitness.values

  #     # Apply mutation on the offspring
  #     for mutant in offspring:
  #         if random.random() < MUTPB:
  #             toolbox.mutate(mutant)
  #             del mutant.fitness.values

  #     # Evaluate the individuals with an invalid fitness
  #     invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
  #     fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
  #     for ind, fit in zip(invalid_ind, fitnesses):
  #         ind.fitness.values = fit

  #     # The population is entirely replaced by the offspring
  #     pop[:] = offspring
  #     hall_of_fame.update(pop)
  #     print(
  #         "\n HALL OF FAME:\n"
  #         + "\n".join(
  #             [
  #                 f"    {_}: {ind}, Fitness: {ind.fitness.values[0]}"
  #                 for _, ind in enumerate(hall_of_fame)
  #             ]
  #         )
  #     )

  #     fitnesses = [
  #         ind.fitness.values[0] for ind in pop if not np.isinf(ind.fitness.values[0])
  #     ]
  #     mean[g] = np.mean(fitnesses)
  #     best[g] = np.max(fitnesses)

  # end_t = time.perf_counter()
  # print(f"Time Elapsed: {end_t - t:,.2f}")

  # OPTIMISED_STRATEGY_PARAMS = {
  #   k: v for k, v in zip(PARAM_NAMES, hall_of_fame[0])}
  # MAE_params = list(OPTIMISED_STRATEGY_PARAMS.values())
  # print('**파라미터 값: ', MAE_params)

# # STC --------------------------------------------------------------------------

#   random.seed(3)

#   PARAM_NAMES = ["period","pfast","pslow","upperLimit","lowerLimit"]

#   NGEN = 5  # 알고리즘 5번 반복.
#   NPOP = 100 #인구 초기
#   CXPB = 0.5  #교차 전략 
#   MUTPB = 0.3  #돌연변이 전략.


#   #최소fintness 설정 (fitness값이 작을수록 좋도록 설정)
#   creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
#   creator.create('Individual', list, fitness=creator.FitnessMin)

#   # creator.create("FitnessMax", base.Fitness, weights=(1.0,))
#   # creator.create("Individual", list, fitness=creator.FitnessMax)

#   def evaluate(individual, plot=False, log=False):

#     strategy_params = {k: v for k, v in zip(PARAM_NAMES, individual)}

#     cerebro = bt.Cerebro(stdstats=False)

#     data = bt.feeds.PandasData(dataname = df_bt, name = i)

#     cerebro.adddata(data)

#     initial_capital = 1000000
#     cerebro.broker.setcash(initial_capital)

#     cerebro.addstrategy((StochasticSR, **strategy_params)

#     cerebro.addanalyzer(bt.analyzers.DrawDown)

#     cerebro.broker.setcommission(commission=0.0025, margin=False)  #수수료 설정

#     strats = cerebro.run()

#     profit = cerebro.broker.getvalue() - initial_capital

#     # max_dd = strats[0].analyzers.drawdown.get_analysis()["max"]["moneydown"] # max.moneydown - max drawdown value in monetary units
#     # fitness = profit / (max_dd if max_dd > 0 else 1)
#     fitness = round(1 / profit, 15)

#     if log:
#       print(f"Starting Portfolio Value: {initial_capital:,.2f}")
#       print(f"Final Portfolio Value:  {cerebro.broker.getvalue():,.2f}")
#       print(f"Total Profit:       {profit:,.2f}")
#       print(f"Maximum Drawdown:     {max_dd:,.2f}")
#       print(f"Profit / Max DD:     {fitness}")

#     # if plot:
#       # cerebro.plot()

#     return [fitness]

#   toolbox = base.Toolbox()
#   toolbox.register("indices", random.sample, range(NPOP), NPOP)

#   # crossover strategy
#   toolbox.register("mate", tools.cxUniform, indpb=CXPB)
#   # mutation strategy
#   toolbox.register("mutate", tools.mutUniformInt, low=1, up=151, indpb=0.2)
#   # selection strategy
#   toolbox.register("select", tools.selTournament, tournsize=3)
#   # fitness function
#   toolbox.register("evaluate", evaluate)


#   # definition of an individual & a population
#   # 파라미터 개수 및 범위 설정 - toolbox.register
#   toolbox.register('attr_period', random.randint, 5, 31) 
#   toolbox.register('attr_pfast', random.randint, 2, 21)
#   toolbox.register('attr_pslow', random.randint, 2, 21)
#   toolbox.register('attr_upperLimit', random.randint, 70, 91)
#   toolbox.register('attr_lowerLimit', random.randint, 10, 31)


#   toolbox.register(
#     "individual",
#     tools.initCycle,
#     creator.Individual,
#     (   # 파라미터 개수 설정
#         toolbox.attr_period,
#         toolbox.attr_pfast,
#         toolbox.attr_pslow,
#         toolbox.attr_upperLimit,
#         toolbox.attr_lowerLimit,
#     ),
#   )

#   toolbox.register("population", tools.initRepeat, list, toolbox.individual)

#   mean = np.ndarray(NGEN)
#   best = np.ndarray(NGEN)
#   hall_of_fame = tools.HallOfFame(maxsize=3)

#   t = time.perf_counter()
#   pop = toolbox.population(n=NPOP)
#   for g in trange(NGEN):
#     # Select the next generation individuals
#     offspring = toolbox.select(pop, len(pop))
#     # Clone the selected individuals
#     offspring = list(map(toolbox.clone, offspring))

#     # Apply crossover on the offspring
#     for child1, child2 in zip(offspring[::2], offspring[1::2]):
#       if random.random() < CXPB:
#         toolbox.mate(child1, child2)
#         del child1.fitness.values
#         del child2.fitness.values

#     # Apply mutation on the offspring
#     for mutant in offspring:
#       if random.random() < MUTPB:
#         toolbox.mutate(mutant)
#         del mutant.fitness.values

#     # Evaluate the individuals with an invalid fitness
#     invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
#     fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
#     for ind, fit in zip(invalid_ind, fitnesses):
#       ind.fitness.values = fit

#     # The population is entirely replaced by the offspring
#     pop[:] = offspring
#     hall_of_fame.update(pop)
#     print(
#       "HALL OF FAME:\n"
#       + "\n".join(
#         [
#           f"  {_}: {ind}, Fitness: {ind.fitness.values[0]}"
#           for _, ind in enumerate(hall_of_fame)
#         ]
#       )
#     )

#     fitnesses = [
#       ind.fitness.values[0] for ind in pop if not np.isinf(ind.fitness.values[0])
#     ]
#     mean[g] = np.mean(fitnesses)
#     best[g] = np.max(fitnesses)

#   end_t = time.perf_counter()
#   print(f"Time Elapsed: {end_t - t:,.2f}")
  
#   # 최적의 파라미터 값 출력
#   OPTIMISED_STRATEGY_PARAMS = {
#     k: v for k, v in zip(PARAM_NAMES, hall_of_fame[0])}
#   STC_params = list(OPTIMISED_STRATEGY_PARAMS.values())
#   print('**STC 파라미터 값: ', STC_params)


  # 데이터 불러와서 가공 ------------------------------------------------------------
  df = pd.read_json(json_data[fullcode_list0[j]], orient ='index') 
  read_df = df.transpose()

  read_df["GDC_sig"] = ""
  read_df["RSI_sig"] = ""
  read_df["ROC_sig"] = ""
  # read_df["MAE_sig"] = ""
  read_df["STC_sig"] = ""

  read_df['TDD_CLSPRC'] = read_df['TDD_CLSPRC'].str.replace(',','').astype('float')
  read_df['pfast'] = talib.MA(read_df['TDD_CLSPRC'], timeperiod = GDC_params[0], matype=0)
  read_df['pslow'] = talib.MA(read_df['TDD_CLSPRC'], timeperiod = GDC_params[1], matype=0)
  read_df['RSI'] = talib.RSI(read_df['TDD_CLSPRC'], timeperiod = RSI_params[0])
  read_df['ROC'] = talib.ROC(read_df['TDD_CLSPRC'], timeperiod = ROC_params[0])
  # read_df['MAE'] = talib.EMA(read_df['TDD_CLSPRC'], timeperiod = MAE_params[0]) # 60일 지수 이동평균
  # read_df['slowk'], read_df['slowd'] = talib.STOCH(read_df['TDD_HGPRC'], read_df['TDD_LWPRC'], read_df['TDD_CLSPRC'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)


  # 매도, 매수 전략 설정 후 GDC_sig 열 추가
  first_cross = 0 
  for i in range(0, len(read_df)):
    if read_df['pfast'][i] < read_df['pslow'][i] and first_cross == 0:
      # print('Death cross on day', df['TRD_DD'][i], ':expect the price to continue to fall (매도)')
      read_df['GDC_sig'][i] = -1
      first_cross=1
    elif read_df['pfast'][i] > read_df['pslow'][i] and first_cross ==1:
      # print('Golden cross on day', df['TRD_DD'][i], ':expect the price to continue to rise (매수)')
      first_cross=0
      read_df['GDC_sig'][i] = 1
    else:
      read_df['GDC_sig'][i] = 0

  # 매도 매수 전략 설정 후 RSI_sig 열 추가
  for i in range(0, len(read_df)):
    if read_df['RSI'][i] < 30: # 30보다 작으면 매수시점
        read_df['RSI_sig'][i] = 1
    elif read_df['RSI'][i] >= 70: # 70보다 크면 매도시점
        read_df['RSI_sig'][i] = -1
    else:
        read_df['RSI_sig'][i] = 0

  # 매도 매수 전략 설정 후 ROC_sig 열 추가
  for i in range(0, len(read_df)):
    if read_df['ROC'][i] < 0: # 30보다 작으면 매도시점
        read_df['ROC_sig'][i] = -1
    elif read_df['ROC'][i] >= 0: # 70보다 크면 매수시점
        read_df['ROC_sig'][i] = 1

  # 매도 매수 전략 설정 후 MAE_sig 열 추가
  
  # 매도 매수 전략 설정 후 STC_sig 열 추가

  # result = read_df.drop(['pfast', 'pslow', 'RSI'], axis='columns')
  result = read_df[['TRD_DD','MKTCAP', 'GDC_sig', 'RSI_sig', 'ROC_sig']]
  print(result)

  # result_js = result.to_json(orient = 'columns') # 데이터프레임을 json 형태로 변환
  # result_dict[fullcode_list0[j]] = result_js # fullcode가 key값이 되고 json이 value값이 됨

# with open('/content/drive/MyDrive/[Shared]K-Project/data/test1.json', 'w') as f:
#     json.dump(result_dict, f)


0
KR7060310000


/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
 20%|██        | 1/5 [02:45<11:03, 165.95s/it]


 HALL OF FAME:
  0: [3, 193], Fitness: -3.1397568572e-05
  1: [67, 200], Fitness: -1.7942111571e-05
  2: [4, 186], Fitness: -1.7186796416e-05


 40%|████      | 2/5 [04:30<06:29, 129.71s/it]


 HALL OF FAME:
  0: [3, 193], Fitness: -3.1397568572e-05
  1: [67, 200], Fitness: -1.7942111571e-05
  2: [4, 186], Fitness: -1.7186796416e-05


...

 80%|████████  | 4/5 [08:23<02:00, 120.75s/it]


 HALL OF FAME:
  0: [37], Fitness: -0.004960932655346
  1: [25], Fitness: -0.00286841161707
  2: [47], Fitness: -0.001375894331315


100%|██████████| 5/5 [10:27<00:00, 125.45s/it]


 HALL OF FAME:
  0: [37], Fitness: -0.004960932655346
  1: [25], Fitness: -0.00286841161707
  2: [47], Fitness: -0.001375894331315
Time Elapsed: 627.26
**ROC 파라미터 값:  [37]


          TRD_DD           MKTCAP GDC_sig RSI_sig ROC_sig
0     2021/12/16  153,621,423,160       0       0        
1     2021/12/15  153,158,708,030       0       0        
2     2021/12/14  156,860,429,070       0       0        
3     2021/12/13  157,554,501,765       0       0        
4     2021/12/10  158,711,289,590       0       0        
...          ...              ...     ...     ...     ...
4856  2002/04/29   16,272,000,000       0       0       1
4857  2002/04/26   18,480,000,000       0       0       1
4858  2002/04/25   21,000,000,000       0       0       1
4859  2002/04/24   20,424,000,000       0       0       1
4860  2002/04/23   18,240,000,000       0       0       1

[4861 rows x 5 columns]


In [ ]:
# 최종적인 저장 형태
print(result)

          TRD_DD           MKTCAP GDC_sig RSI_sig ROC_sig
0     2021/12/16  153,621,423,160       0       0        
1     2021/12/15  153,158,708,030       0       0        
2     2021/12/14  156,860,429,070       0       0        
3     2021/12/13  157,554,501,765       0       0        
4     2021/12/10  158,711,289,590       0       0        
...          ...              ...     ...     ...     ...
4856  2002/04/29   16,272,000,000       0       0       1
4857  2002/04/26   18,480,000,000       0       0       1
4858  2002/04/25   21,000,000,000       0       0       1
4859  2002/04/24   20,424,000,000       0       0       1
4860  2002/04/23   18,240,000,000       0       0       1

[4861 rows x 5 columns]


In [ ]:
# 제대로 저장되었는지 테스트
# with open('/content/drive/MyDrive/[Shared]K-Project/data/test1.json',"r") as json_file:
#     json_data = json.load(json_file) # json file open

df = pd.read_json(json_data['KR7060310000'], orient ='index') # 첫번째 키값으로 데이터프레임 변환
json_df = df.transpose()
print(json_df.columns)

Index(['TRD_DD', 'ISU_CD', 'ISU_NM', 'TDD_CLSPRC', 'TDD_OPNPRC', 'TDD_HGPRC',
       'TDD_LWPRC', 'MKTCAP', 'ACC_TRDVOL', 'EPS', 'PER', 'BPS', 'PBR', 'DPS',
       'DVD_YLD'],
      dtype='object')


In [ ]:
# json 파일 합치는 방법
# https://great-woman-hoseung.tistory.com/6